# Inaugural classification models.

### Based on features from Omnidian database 101, we compare K-Nearest Neighbors, Gradient Boosting, Random Forest, Bagging, and Logistic Regression.

In [122]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn import tree
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.utils import resample
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-paper')

In [2]:
from omnotes import get_preds, score_model, human_readify

In [29]:
import pickle

Get Data

In [3]:
data101 = pd.read_csv('data/eda101.csv')
data101.head()

,Ticket_Id,Asset_Id,asset_id,Resolution_Code,Root_Cause,Ticket_Creation_Reason,asset_type,latitude,longitude,tilt,azimuth,Ticket_Status,Ticket_Origin,Service_Type,Service_Partner,Ticket_Tags,Date_Ticket_Initially_Assigned,Date_Ticket_Closed,installed_by,installation_date
0,23947,101112604,101112604,inspect system,root_cause_normal_wear_and_tear,communication offline,Residential Solar PV,38.944642,-121.248833,37.0,190.0,Closed,origin_omnidian_customer,Field Service,SunUp STS Service Team,"comment_from_another_agent,communication_statu...",2018-10-22 17:25:27,2018-11-29,Williams Lifetime Builders Inc. DBA Lifetime S...,2013-04-04 00:00:00
1,23947,101112604,101112604,inspect system,root_cause_normal_wear_and_tear,communication offline,Residential Solar PV,38.944642,-121.248833,37.0,190.0,Closed,origin_omnidian_customer,Field Service,SunUp STS Service Team,"comment_from_another_agent,communication_statu...",2018-10-22 17:25:27,2018-11-29,Williams Lifetime Builders Inc. DBA Lifetime S...,2013-04-04 00:00:00
2,27384,101113056,101113056,inspect system,root_cause_normal_wear_and_tear,system inspection,Residential Solar PV,34.101697,-118.146646,18.0,180.0,Closed,origin_homeowner,Field Service,IndaSpec Solar Service Team,"comment_from_another_agent,comment_from_end-us...",2018-12-20 06:49:02,2019-02-09,Green Tech Solutions Inc.,2014-04-08 00:00:00
3,27384,101113056,101113056,inspect system,root_cause_normal_wear_and_tear,system inspection,Residential Solar PV,34.101697,-118.146646,18.0,180.0,Closed,origin_homeowner,Field Service,IndaSpec Solar Service Team,"comment_from_another_agent,comment_from_end-us...",2018-12-20 06:49:02,2019-02-09,Green Tech Solutions Inc.,2014-04-08 00:00:00
4,27384,101113056,101113056,inspect system,root_cause_normal_wear_and_tear,system inspection,Residential Solar PV,34.101697,-118.146646,18.0,90.0,Closed,origin_homeowner,Field Service,IndaSpec Solar Service Team,"comment_from_another_agent,comment_from_end-us...",2018-12-20 06:49:02,2019-02-09,Green Tech Solutions Inc.,2014-04-08 00:00:00


In [4]:
data101.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 861 entries, 0 to 860
Data columns (total 20 columns):
Ticket_Id                         861 non-null int64
Asset_Id                          861 non-null int64
asset_id                          861 non-null int64
Resolution_Code                   861 non-null object
Root_Cause                        861 non-null object
Ticket_Creation_Reason            861 non-null object
asset_type                        861 non-null object
latitude                          861 non-null float64
longitude                         861 non-null float64
tilt                              861 non-null float64
azimuth                           861 non-null float64
Ticket_Status                     861 non-null object
Ticket_Origin                     861 non-null object
Service_Type                      861 non-null object
Service_Partner                   861 non-null object
Ticket_Tags                       861 non-null object
Date_Ticket_Initially_Assigned

Assign target to Root_Cause and Train-Test-Split

In [5]:
X = data101.drop('Root_Cause', axis=1)
y = data101['Root_Cause']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

One hot encoding: since our data is largely categorical but our models require numeric inputs, we one hot encode it.


In [6]:
encoder = OneHotEncoder(categories='auto', handle_unknown='ignore')
encoder.fit(X_train)

OneHotEncoder(categorical_features=None, categories='auto', drop=None,
              dtype=<class 'numpy.float64'>, handle_unknown='ignore',
              n_values=None, sparse=True)

In [7]:
X_train = pd.DataFrame(encoder.transform(X_train).toarray(),
                         columns=encoder.get_feature_names())
X_test = pd.DataFrame(encoder.transform(X_test).toarray(),
                        columns=encoder.get_feature_names())

Training our models.

In [8]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [34]:
filename = '101_knn.pkl'
pickle.dump(knn, open(filename, 'wb'))

In [9]:
lr = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [35]:
filename = '101_lr.pkl'
pickle.dump(lr, open(filename, 'wb'))

In [27]:
gb = GradientBoostingClassifier(random_state=42, min_samples_leaf=30)
gb.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=30, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=42, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [36]:
filename = '101_gb.pkl'
pickle.dump(gb, open(filename, 'wb'))

In [37]:
dt = DecisionTreeClassifier(random_state=42, min_samples_leaf=30)
dt.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=30, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best')

In [41]:
filename = '101_dt.pkl'
pickle.dump(dt, open(filename, 'wb'))

In [39]:
bg = BaggingClassifier(random_state=25565)
bg.fit(X_train, y_train)

BaggingClassifier(base_estimator=None, bootstrap=True, bootstrap_features=False,
                  max_features=1.0, max_samples=1.0, n_estimators=10,
                  n_jobs=None, oob_score=False, random_state=25565, verbose=0,
                  warm_start=False)

In [42]:
filename = '101_bg.pkl'
pickle.dump(bg, open(filename, 'wb'))

In [43]:
rf = RandomForestClassifier(random_state=42, n_estimators=100)
rf.fit(X_train, y_train);

In [44]:
filename = '101_rf.pkl'
pickle.dump(rf, open(filename, 'wb'))

Let's show our results

In [25]:
knn.score(X_test, y_test)

0.6820809248554913

In [26]:
lr.score(X_test, y_test)

0.9017341040462428

In [28]:
gb.score(X_test, y_test)

0.7456647398843931

In [45]:
dt.score(X_test, y_test)

0.6936416184971098

In [46]:
bg.score(X_test, y_test)

0.9190751445086706

In [47]:
rf.score(X_test, y_test)

0.9075144508670521

What may be some other classifiers?

In [48]:
from sklearn.utils.testing import all_estimators

In [56]:
rf.predict_proba(X_test)[0]

array([0.  , 0.  , 0.  , 0.01, 0.  , 0.11, 0.  , 0.88, 0.  , 0.  ])

In [60]:
rf.predict(X_test)[0:5]

array(['root_cause_normal_wear_and_tear',
       'root_cause_major_component_failure_warranty',
       'root_cause_normal_wear_and_tear',
       'root_cause_normal_wear_and_tear',
       'root_cause_major_component_failure_warranty'], dtype=object)

In [94]:
l_props = lr.predict_proba(X_test)[1]

In [100]:
zipp = list(zip(lr.classes_, l_props))

In [101]:
zipp

[('root_cause_design/sale_issue', 0.00043387759236169344),
 ('root_cause_environmental', 0.004206441640046431),
 ('root_cause_homeowner', 0.014217294259570568),
 ('root_cause_installer_workmanship', 0.004648355311296379),
 ('root_cause_major_component_failure_non-warranty', 0.0036398455245150647),
 ('root_cause_major_component_failure_warranty', 0.9000584314884308),
 ('root_cause_non-service_support', 0.027058029061024055),
 ('root_cause_normal_wear_and_tear', 0.0411124777776664),
 ('root_cause_roof_issue', 0.0027033523841786256),
 ('root_cause_service_workmanship', 0.0019218949609099132)]

In [110]:
#want a dictionary that gives probability for each class, 
#prediction, and ground truth
def display_preds_truth(model, obs, X_test, y_test):
    probs = model.predict_proba(X_test)[obs]
    classes = model.classes_
    display = dict(zip(classes, probs))
#     display['prediction'] = model.predict(X_test)[obs]
    display['ground truth'] = y_test[obs]
    return display
        
    
    

In [129]:
#want a dictionary that gives probability for each class,
def display_probas(model, obs, X_test):
    probs = model.predict_proba(X_test)[obs]
    classes = model.classes_
    display = dict(zip(classes, probs))
    display['prediction'] = model.predict(X_test)[obs]
    return display

In [130]:
display_probas(lr, 1, X_test)

{'root_cause_design/sale_issue': 0.00043387759236169344,
 'root_cause_environmental': 0.004206441640046431,
 'root_cause_homeowner': 0.014217294259570568,
 'root_cause_installer_workmanship': 0.004648355311296379,
 'root_cause_major_component_failure_non-warranty': 0.0036398455245150647,
 'root_cause_major_component_failure_warranty': 0.9000584314884308,
 'root_cause_non-service_support': 0.027058029061024055,
 'root_cause_normal_wear_and_tear': 0.0411124777776664,
 'root_cause_roof_issue': 0.0027033523841786256,
 'root_cause_service_workmanship': 0.0019218949609099132,
 'prediction': 'root_cause_major_component_failure_warranty'}

In [124]:
y_test

714                    root_cause_normal_wear_and_tear
605        root_cause_major_component_failure_warranty
120                    root_cause_normal_wear_and_tear
208                    root_cause_normal_wear_and_tear
380        root_cause_major_component_failure_warranty
816        root_cause_major_component_failure_warranty
575                    root_cause_normal_wear_and_tear
266        root_cause_major_component_failure_warranty
557                    root_cause_normal_wear_and_tear
825                    root_cause_normal_wear_and_tear
580                    root_cause_normal_wear_and_tear
327                    root_cause_normal_wear_and_tear
623    root_cause_major_component_failure_non-warranty
365        root_cause_major_component_failure_warranty
827                     root_cause_non-service_support
198                    root_cause_normal_wear_and_tear
456                    root_cause_normal_wear_and_tear
465                    root_cause_normal_wear_and_tear
311       